In [1]:
import requests
import concurrent

import aiohttp
import asyncio
import nest_asyncio #Do this if you want to run it on JN
nest_asyncio.apply()

#Results may vary because of network issues. 
#Running this 100 times should give us a better statistics but ain't nobody got time for that

In [2]:
%%time
#Sequential processing - retrieve 150 pokemons

pokemons_sequential = []

#For 50 pokemons, run the request and append it to the list
for number in range(1, 151):
    pokemon_url = f'https://pokeapi.co/api/v2/pokemon/{number}'
    response = requests.get(pokemon_url)
    pokemons_sequential.append(response.json().get('name'))

print(len(pokemons_sequential))
print(pokemons_sequential[10])

150
metapod
CPU times: user 4.79 s, sys: 454 ms, total: 5.24 s
Wall time: 19 s


In [3]:
%%time
#Multithreading

#Create a function that will parallelized
def request_pokemon(number:int) -> str:
    response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{number}')
    return response.json().get('name')

#Set a threadpoool executor with 6 cores and run the function for the list of parameters
with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
    res = [executor.submit(request_pokemon, number) for number in range(1, 151)]
    concurrent.futures.wait(res)
    pokemons_parallel = [r.result() for r in res] #Gather up the results in a list
    
print(len(pokemons_parallel))
print(pokemons_parallel[10])

150
metapod
CPU times: user 1.84 s, sys: 158 ms, total: 2 s
Wall time: 3.07 s


In [4]:
%%time
#Async processing
#We are using nest asyncio to be able to run it on jupyter notebook: 
#https://github.com/erdewit/nest_asyncio
pokemons_async = []

async def get_pokemon(session, url):
    async with session.get(url) as resp:
        pokemon = await resp.json()
        return pokemon.get('name')

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for number in range(1, 151):
            url = f'https://pokeapi.co/api/v2/pokemon/{number}'
            tasks.append(asyncio.ensure_future(get_pokemon(session, url)))
        original_pokemon = await asyncio.gather(*tasks)
        for pokemon in original_pokemon:
            pokemons_async.append(pokemon)

asyncio.run(main())

print(len(pokemons_async))
print(pokemons_async[10])

150
metapod
CPU times: user 522 ms, sys: 42.5 ms, total: 564 ms
Wall time: 1.53 s
